# What makes Julia fast?

## Multiple Dispatch

```
function sort(x::Array{String, 1})
    ...
end

function sort(x::Array{Int, 1})
    ...
end

function sort(x::Array{Bool, 1})
    ...
end
```

We need a function, f, to square its input and then compute its value mod 4. Assume speed is important.

We always need f to output an integer, but its input, x, can be a String, Float64, or Int64, and we won’t know the type of x until runtime.

In [12]:
f(x::Int64) = x^2 % 4
f(x::Float64) = f(ceil(Int64, x))
f(x::String) = f(parse(Float64, x))

f (generic function with 3 methods)

In [26]:
function applytodata(array)
    for a in array
        f(a)
    end
end;

array = [s for i=1:1000000 for s in ["5.5",6,5.5]]

applytodata(array);

println("Dispatch Speed")
@time applytodata(array);

Dispatch Speed
  0.097743 seconds


## Compiler Efficiency

The compiler translates dispatched functions into extremely efficient LLVM code, optimized for the each specific type. In fact, the Julia compiler will even remove lines of code for you 

In [29]:
function bigger_floats(x::Real)
    return tentimes(x)
end
tentimes(x::AbstractFloat) = 10.0x
tentimes(x::Real) = false

tentimes (generic function with 2 methods)

In [30]:
@code_llvm bigger_floats(10.0)


;  @ In[29]:1 within `bigger_floats'
define double @julia_bigger_floats_1709(double) {
top:
;  @ In[29]:2 within `bigger_floats'
; ┌ @ In[29]:4 within `tentimes'
; │┌ @ float.jl:405 within `*'
    %1 = fmul double %0, 1.000000e+01
; └└
  ret double %1
}


In [32]:
@code_llvm bigger_floats(1)


;  @ In[29]:1 within `bigger_floats'
define i8 @julia_bigger_floats_1712(i64) {
top:
;  @ In[29]:2 within `bigger_floats'
  ret i8 0
}


## Another example

Write a program to count the number of lines of code in a file

```
def countlines(fname):
    fstream = open(fname)
    num_lines = len(fstream.readlines())
    fstream.close()
    return num_lines
```

```
function countlines(fname::String)
    fstream = open(fname)
    num_lines = length(readlines(fstream))
    close(fstream)
    return num_lines
end
```

Modify this function to handle a single file, a list of files, and have the capacity to filter files by extension.

### Python

```
def countlines(*args):
    if len(args) == 1:
        files = args[0]
        if isinstance(files, list):
            return sum([filelines(f) for f in files])
        if isinstance(files, str):
            return filelines(files)
        else:
            raise Exception("Unsupported data format")
    if len(args) == 2:
        files, ext = args
        if isinstance(files, list):
            filtered = [f for f in files if f.endswith(ext)]
            return sum([filelines(f) for f in filtered])
        if isinstance(files, str):
            if files.endswith(ext):
                return filelines(files)
            else:
                return 0
        else:
            raise Exception("Unsupported data format")
    else:
        raise Exception("Unsupported data format")
        
def filelines(file):
    fstream = open(fname)
    num_lines = len(fstream.readlines())
    fstream.close()
    return num_lines
```

### Julia

```
function countlines(fname::String)
    fstream = open(fname)
    num_lines = length(readlines(fstream))
    close(fstream)
    return num_lines
end

function countlines(files::Array{String, 1})
    return sum(map(countlines, files))
end

function countlines(files::Array{String, 1}, ext::String)
    filtered = filter(x->endswith(x, ext), files)
    return countlines(filtered) 
end

function countlines(file::String, ext::String)
    return endswith(file, ext) ? countlines(file) : 0
end
```

In [1]:
struct MyStruct{T} end

In [2]:
function f(s::MyStruct{1}) println("f1") end
function f(s::MyStruct{2}) println("f2") end
function f(s::MyStruct) println("f3") end

f (generic function with 3 methods)

In [3]:
f(MyStruct{1}())
f(MyStruct{2}())
f(MyStruct{3}())

f1
f2
f3


In [8]:
x = 2
f(MyStruct{x}())

f2
